In [6]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import mlab
import scipy.io
from scipy import signal
from scipy.signal import firwin
from scipy.signal import welch
from scipy.signal import spectrogram
import scipy.signal
from scipy import stats
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
from sklearn.model_selection import train_test_split,KFold  # Add this import
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

In [7]:
# List of MATLAB files
mat_files = [
    "P1_pre_training.mat",
    "P1_pre_test.mat",
    "P1_post_training.mat",
    "P1_post_test.mat",
    "P2_pre_training.mat",
    "P2_pre_test.mat",
    "P2_post_training.mat",
    "P2_post_test.mat",
    "P3_pre_training.mat",
    "P3_pre_test.mat",
    "P3_post_training.mat",
    "P3_post_test.mat",
]

In [8]:
# Initialize variables for data storage
eeg_data = []

# PRE - PROCESSING

In [30]:
# Loop through the MATLAB files
for file in mat_files:
    mat_data = scipy.io.loadmat(file)
    
    
    # Extract EEG data (without labels)
    y = y_with_labels[1:, :].astype(float)  # Remove the first row containing labels and convert to float
    fs = mat_data.get('fs')[0, 0]
    
    # Convert channel names from numpy array to strings
    channel_names = [str(element[0]) for element in y_with_labels[0, :]]
    
    # Create an MNE Raw object with channel names and data
    info = mne.create_info(ch_names=channel_names, sfreq=fs)
    raw = mne.io.RawArray(y.T, info)
    
    # Define the desired filter parameters
    l_freq = 1  # Low-pass frequency
    h_freq = None  # High-pass frequency (None for no high-pass filter)
    
    # Calculate the power in each channel and time point
    power = np.sum(np.square(y), axis=0)
    
    # Check if there is any non-zero power (frequency components)
    if np.any(power):
        # If there are frequencies, apply the filter to EEG channels
        
        # Manually specify EEG channel names
        eeg_channel_names = ['EEG 1', 'EEG 2', 'EEG 3', 'EEG 4', 'EEG 5', 'EEG 6', 'EEG 7', 'EEG 8', 'EEG 9', 'EEG 10', 'EEG 11', 'EEG 12', 'EEG 13', 'EEG 14', 'EEG 15', 'EEG 16']

        # Use the channel names to create picks
        eeg_picks = [raw.info['ch_names'].index(name) for name in eeg_channel_names]

        raw.filter(l_freq=l_freq, h_freq=h_freq, picks=eeg_picks)
        
        # Reference electrodes (choose an appropriate reference)
        raw.set_eeg_reference(ref_channels='average')
        
        # Identify and mark bad channels (modify this based on your data quality)
        # raw.info['bads'] = ['EEG 1', 'EEG 2']  # Example: Mark channels as bad
        
        # Re-sample the data to a common sampling rate if needed
        target_sfreq = 256  # Adjust to your desired sampling rate
        raw.resample(target_sfreq)
        
        eeg_data.append(raw)
    else:
        # Handle the case where there are no frequency components
        print(f"No frequencies found in {file}. Skipping preprocessing.")

Creating RawArray with float64 data, n_channels=16, n_times=271816
    Range : 0 ... 271815 =      0.000 ...  1061.777 secs
Ready.
No data channels found. The highpass and lowpass values in the measurement info will not be updated.
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 845 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    0.1s finished


ValueError: No EEG, ECoG, sEEG or DBS channels found to rereference.

In [29]:
print(raw.info['ch_names'])

['EEG 1', 'EEG 2', 'EEG 3', 'EEG 4', 'EEG 5', 'EEG 6', 'EEG 7', 'EEG 8', 'EEG 9', 'EEG 10', 'EEG 11', 'EEG 12', 'EEG 13', 'EEG 14', 'EEG 15', 'EEG 16']
eeg_picks


# Data Visualization: 
The preprocessed data can be used for basic data visualization.

# Frequency Analysis: 
You will need to implement specific methods for frequency analysis (e.g., FFT, multitaper) and adjust the code accordingly.

# Event-Related Potentials (ERPs): 
You can create epochs from the preprocessed data, but ERP analysis often involves additional steps such as baseline correction and averaging.

# Time-Frequency Analysis: 
You can perform time-frequency analysis using techniques like Morlet wavelets, but you may need to customize the code for more advanced analyses.

# Artifact Removal: 
The code includes basic preprocessing, but additional artifact detection and removal methods may be required.

# Statistical Analysis: 
The code includes basic statistical testing, but specific tests and analyses will depend on your research questions.

# Connectivity Analysis: 
Connectivity analysis typically requires preprocessing steps specific to connectivity measures (e.g., coherence, cross-correlation).

# Machine Learning Classification: 
Machine learning requires feature extraction and model training, which are not included in the preprocessing code.

# Source Localization: 
Source localization involves complex modeling and may require specialized preprocessing steps.

# Temporal and Spatial Patterns: 
Analyzing temporal and spatial patterns depends on your specific research goals and analysis techniques.

# Data Visualization